In [93]:
import yahoo_fin.stock_info as si
import pandas as pd
import yahoo_fin.options as ops
from requests_html import HTMLSession
from datetime import datetime

In [94]:
%pip install lxml_html_clean


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [95]:
si.get_data('AAPL')

,open,high,low,close,adjclose,volume,ticker
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.098834,469033600,AAPL
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093678,175884800,AAPL
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086802,105728000,AAPL
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.088951,86441600,AAPL
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091530,73449600,AAPL
...,...,...,...,...,...,...,...
2024-12-20,248.039993,255.000000,245.690002,254.490005,254.490005,147495300,AAPL
2024-12-23,254.770004,255.649994,253.449997,255.270004,255.270004,40858800,AAPL
2024-12-24,255.490005,258.209991,255.289993,258.200012,258.200012,23234700,AAPL
2024-12-26,258.190002,260.100006,257.630005,259.019989,259.019989,27263000,AAPL


In [96]:
def parse_dates(date_strings):
    """
    Converts a list of date strings in the format 'Dec 27, 2024' to Python datetime.date objects.
    Ignores any strings that do not match the format.

    Args:
        date_strings (list of str): List of date strings to parse.

    Returns:
        list of datetime.date: List of valid datetime.date objects.
    """
    parsed_dates = []
    date_format = "%b %d, %Y"  # Format corresponding to 'Dec 27, 2024'

    for date_str in date_strings:
        try:
            # display(date_str)
            parsed_date = datetime.strptime(date_str.strip(), date_format).date()
            parsed_dates.append(parsed_date)
        except ValueError:
            # Ignore strings that do not match the expected format
            continue

    return parsed_dates

def build_options_url(ticker, date = None):
    """Constructs the URL pointing to options chain"""
    url = "https://finance.yahoo.com/quote/" + ticker + "/options?p=" + ticker
    if date is not None:
        url = url + "&date=" + str(int(pd.Timestamp(date).timestamp()))
    return url

def get_expiration_dates(ticker):
    """Scrapes the expiration dates from each option chain for input ticker
       @param: ticker"""
    site = build_options_url(ticker)
    session = HTMLSession()
    resp = session.get(site)
    dates_str = [x.text for x in resp.html.lxml.find_class("itm")]
    dates = parse_dates(dates_str)
    session.close()
    return dates

In [97]:
# %%capture --no-display
# ticker = "AAPL"
# expiration_dates = get_expiration_dates("AAPL")
# chains = { d: ops.get_options_chain(ticker, d) for d  in expiration_dates  }

In [98]:
chains.keys()

dict_keys([datetime.date(2024, 12, 27), datetime.date(2025, 1, 3), datetime.date(2025, 1, 10), datetime.date(2025, 1, 17), datetime.date(2025, 1, 24), datetime.date(2025, 1, 31), datetime.date(2025, 2, 21), datetime.date(2025, 3, 21), datetime.date(2025, 4, 17), datetime.date(2025, 6, 20), datetime.date(2025, 7, 18), datetime.date(2025, 8, 15), datetime.date(2025, 9, 19), datetime.date(2025, 12, 19), datetime.date(2026, 1, 16), datetime.date(2026, 6, 18), datetime.date(2026, 12, 18), datetime.date(2027, 1, 15)])

In [107]:
for d in chains.keys():
    chains[d]['calls']['Type'] = 'call'
    chains[d]['calls']['Expiry'] = d
    chains[d]['puts']['Type'] = 'put'
    chains[d]['puts']['Expiry'] = d

optionsFrame = pd.concat([chains[c]['calls'] for c in chains], axis=0)
optionsFrame

,Contract Name,Last Trade Date (EDT),Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,Expiry,Type
0,AAPL241227C00110000,12/18/2024 8:54 PM,110.0,139.70,144.55,145.50,0.00,0.00%,-,4,0.00%,2024-12-27,call
1,AAPL241227C00120000,12/27/2024 4:14 PM,120.0,134.25,134.65,135.65,-4.52,-3.26%,1,1,50.00%,2024-12-27,call
2,AAPL241227C00125000,12/27/2024 4:56 PM,125.0,128.70,129.50,130.60,6.61,5.41%,2,0,0.00%,2024-12-27,call
3,AAPL241227C00130000,12/23/2024 4:02 PM,130.0,124.01,124.65,125.55,0.00,0.00%,4,4,0.00%,2024-12-27,call
4,AAPL241227C00135000,12/27/2024 5:11 PM,135.0,118.40,119.55,120.60,-5.27,-4.26%,1,4,0.00%,2024-12-27,call
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,AAPL270115C00410000,12/26/2024 4:02 PM,410.0,5.30,4.40,5.40,-0.15,-2.75%,1,62,25.47%,2027-01-15,call
57,AAPL270115C00420000,12/27/2024 4:24 PM,420.0,4.40,3.85,4.60,0.12,2.80%,3,3254,25.28%,2027-01-15,call
58,AAPL270115C00430000,12/24/2024 3:12 PM,430.0,4.00,2.89,4.30,0.00,0.00%,1,22,25.67%,2027-01-15,call
59,AAPL270115C00440000,12/27/2024 5:52 PM,440.0,3.30,3.00,3.55,-0.20,-5.71%,12,199,25.30%,2027-01-15,call
